In [ ]:
# non_linear_model
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,dataset_gslf)
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)

modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)



In [ ]:
def prepare_ravia_similarity_mols2(df_ravia_similarity_mean,lm,tokenizer,modeldeepchem_gslf):
    df_ravia_mean_mols1 = df_ravia_similarity_mean[['Stimulus 1-IsomericSMILES','Stimulus 1-nonStereoSMILES','CID Stimulus 1']].drop_duplicates().reset_index(drop=True)
    df_ravia_mean_mols2 = df_ravia_similarity_mean[['Stimulus 2-IsomericSMILES','Stimulus 2-nonStereoSMILES','CID Stimulus 2']].drop_duplicates().reset_index(drop=True).rename(columns={'Stimulus 2-nonStereoSMILES': 'Stimulus 1-nonStereoSMILES','Stimulus 2-IsomericSMILES':'Stimulus 1-IsomericSMILES', 'CID Stimulus 2': 'CID Stimulus 1' })
    df_ravia_mols= pd.concat([df_ravia_mean_mols1, df_ravia_mean_mols2], ignore_index=True, axis=0).reset_index(drop=True)
    df_ravia_mols=df_ravia_mols.drop_duplicates().reset_index(drop=True)
    df_ravia_mols = df_ravia_mols.rename(columns={'Stimulus 1-IsomericSMILES': 'IsomericSMILES','Stimulus 1-nonStereoSMILES':'nonStereoSMILES', 'CID Stimulus 1': 'CID' })
    mol_type="nonStereoSMILES"


    res=prepare_mols_helper(lm,tokenizer,df_ravia_mols,modeldeepchem=modeldeepchem_gslf)
    if modeldeepchem_gslf is not None:
        df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored=res
        return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored
    else:
        df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored=res
        return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored
    # return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored


In [ ]:
# df_ravia_similarity_mols,df_ravia_similarity_mols_embeddings_original,df_ravia_similarity_mols_layers_original\
# ,df_ravia_similarity_mols_embeddings,df_ravia_similarity_mols_embeddings_zscored,\
# df_ravia_similarity_mols_layers,df_ravia_similarity_mols_layers_zscored,df_ravia_similarity_mols_embeddings_linear\
# ,df_ravia_similarity_mols_embeddings_linear_zscored=prepare_ravia_similarity_mols2(df_ravia_similarity_mean,lm,tokenizer,modeldeepchem_gslf=None)

In [ ]:
def cosine_sim_helper(df_mols_embeddings,df_mols_embeddings_zscored,df_mols_layers,df_mols_layers_zscored,df_mols_embeddings_linear,df_mols_embeddings_linear_zscored):
    
    cosine_sim_df_mols_embeddings=cosine_similarity_df(df_mols_embeddings,'Combined')
    cosine_sim_df_mols_embeddings_zscored=cosine_similarity_df(df_mols_embeddings_zscored,'Combined')
    
    
    cosine_sim_df_mols_embeddings_linear=cosine_similarity_df(df_mols_embeddings_linear,'Combined')
    cosine_sim_df_mols_embeddings_linear_zscored=cosine_similarity_df(df_mols_embeddings_linear_zscored,'Combined')
    
    ### Cosine similarity for all layers
    cosine_sim_df_mols_layers = []
    cosine_sim_df_mols_layers_zscored = []
    # embedding = molecules_activations_embeddings_original[0]
    for embeddings in df_mols_layers:
        cosine_sim_df_mols_layers.append(cosine_similarity_df(embeddings,'Combined'))
        
    for embeddings in df_mols_layers_zscored:
        cosine_sim_df_mols_layers_zscored.append(cosine_similarity_df(embeddings,'Combined'))
        
    return cosine_sim_df_mols_embeddings, cosine_sim_df_mols_embeddings_zscored, cosine_

In [ ]:
### SOME FMRI USECASES

In [ ]:
for subject in [1,2,3]:
    for roi in ['PirF', 'PirT','AMY']:
        selected_subject=df_sagar_fmri_molformer[(df_sagar_fmri_molformer['subject']==subject)][[roi,'CID','IsomericSMILES','nonStereoSMILES', 'Combined']]
        fmri_data=selected_subject[roi]
        chemical_data=selected_subject['Combined']
        # print(fmri_data)
        fmri_data=convert_fmri_data(fmri_data)
        chemical_data=convert_chemical_data(chemical_data)
        print(chemical_data.shape)
        print(fmri_data.shape)
        corrs, acc, acc_std, all_preds, all_test_data,pearson,res1=run_class_time_CV_fmri_crossval_ridge(chemical_data,fmri_data)
        print(np.max(res1))
        print("rr",res1)

In [ ]:
def convert_chemical_data(data_list):
    datas=[]

    for data in data_list:
        array_str=data
        datas.append(array_str)
    # datas.shape
    return np.asarray(datas)

In [ ]:
def convert_fmri_data(data_list):
    datas=[]

    for data in data_list:
        array_str=data
        array_str = array_str.replace("[array(", "").replace("\n", "").replace(")]", "")
        array_np = np.array(ast.literal_eval(array_str))
        datas.append(array_np)
    # datas.shape
    return np.asarray(datas)

In [ ]:
def prepare_sagar_fmri_mols():

    input_file_sagar = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_sagar_flattened_fmri_mols.csv'
    df_sagar=pd.read_csv(input_file_sagar)
    df_sagar = df_sagar.rename(columns={"cid":"CID"})
    df_sagar = df_sagar.reset_index()
    df_sagar_mols_embeddings_original,df_sagar_mols_layers_original,df_sagar_mols_embeddings,df_sagar_mols_embeddings_zscored,df_sagar_mols_layers,df_sagar_mols_layers_zscored,df_sagar_mols_embeddings_linear,df_sagar_mols_embeddings_linear_zscored=prepare_mols_helper(modeldeepchem_gslf,df_sagar,mol_type="nonStereoSMILES",index=["CID","subject"])
    return df_sagar,df_sagar_mols,df_sagar_mols_embeddings_original,df_sagar_mols_layers_original,df_sagar_mols_embeddings,df_sagar_mols_embeddings_zscored,df_sagar_mols_layers,df_sagar_mols_layers_zscored,df_sagar_mols_embeddings_linear,df_sagar_mols_embeddings_linear_zscored


    # return df_sagar




In [ ]:
df_sagar,df_sagar_mols,df_sagar_mols_embeddings_original,df_sagar_mols_layers_original,df_sagar_mols_embeddings,df_sagar_mols_embeddings_zscored,df_sagar_mols_layers,df_sagar_mols_layers_zscored,df_sagar_mols_embeddings_linear,df_sagar_mols_embeddings_linear_zscored=prepare_sagar_fmri_mols()
# df_mols_embeddings.head(5)


In [ ]:
df_sagar_fmri_molformer = pd.merge(df_sagar_mols_embeddings_zscored, df_sagar,  how='left', left_on=['subject','CID'], right_on = ['subject','CID'])

In [ ]:
def metrics(y_true,y_predicted_prob):
    y_pred = np.round(y_predicted_prob)
    average_type = [None, 'macro' , 'weighted']

    # for average in average_type:
    ap = average_precision_score(y_true, y_predicted_prob,average=None)
    ap_macro = average_precision_score(y_true, y_predicted_prob,average='macro')
    ap_weighted = average_precision_score(y_true, y_predicted_prob,average='weighted')

    precision = precision_score(y_true, y_pred,average=None)
    precision_macro = precision_score(y_true, y_pred,average='macro')
    precision_weighted = precision_score(y_true, y_pred,average='weighted')


    recall = recall_score(y_true, y_pred,average=None)
    recall_macro = recall_score(y_true, y_pred,average='macro')
    recall_weighted = recall_score(y_true, y_pred,average='weighted')


    f1 = f1_score(y_true, y_pred,average=None)
    f1_macro = f1_score(y_true, y_pred,average='macro')
    f1_weighted = f1_score(y_true, y_pred,average='weighted')


    # true_labels = np.round(y_true)

    # print(true_labels)
    # print(y_true)
    return ap_macro,precision_macro,recall_macro,f1_macro

In [ ]:
# df_corrs=pd.DataFrame.from_dict(data, orient='index',
#                            columns=['Dataset','type', 'Correlation'])
# sns.set_style("white")
# fig, ax = plt.subplots(figsize=(7.5, 7.5)
#                        # ,constrained_layout = True
#                       )
# # sns.set_style("whitegrid")
# # sns.color_palette("tab10")
# sns.color_palette("hls", 4)
# palette=[ "#91bfdb",
#             "#003f5c",
#              # "#d73027",
#              # "#fc8d59",
#              # "#ffda33",

#             # , "#4575b4"
# ]

# palette=['#4d79a4','#ecc947','#b07aa0']

# # plt.subplots_adjust(bottom=0.3)
# g=sns.barplot(df_corrs, x="Dataset", y="Correlation", hue= "type",width=0.4,palette=palette)
# # for i in ax.containers:
# #     ax.bar_label(i,fmt="%2.2f")
# # ax.legend().set_title(title)
# handles, labels = ax.get_legend_handles_labels()
# ax.get_legend().remove()
# fig.subplots_adjust(bottom=0.35,left=0.2)
# fig.legend(handles, labels, ncol=1, columnspacing=1, prop={'size': 25}, handlelength=1.5, loc="lower center",
#            borderpad=0.3,

#            bbox_to_anchor=(0.54, 0.),

#            frameon=True, labelspacing=0.4,handletextpad=0.2)
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

# # g.set_yticks([0.45,0.5,0.55,0.6,0.65,0.7]) # <--- set the ticks first
# # g.set_yticklabels(['', '','', '','', ''])

# ax.set_ylabel('Correlation Coefficient')

# # plt.margins(x=0.5)
# # fig.update_layout(bargap=0.1)
# # ax.set_subtitle("Dataset")  # Set title with numeric distance
# # axes[0].set_title("Total Bill Distribution", loc=(0.05, 0.9))  # Set title with numeric distance

# # ax.set_title('')
# ax.xaxis.set_label_coords(0.5, -0.13)
#     # plt.tight_layout()
# #  g.set_yticks([0.45,0.5,0.55,0.6,0.65,0.7]) # <--- set the ticks first

# #  g.set_yticklabels(['', '0.5','', '0.6','', '0.7'])
# #  g.set_ylim(0.45,0.7)

# plt.savefig(filename,
#                 # bbox_inches="tight"
#                )

In [ ]:
# def create_linear_classifier(TASKS, n_classes,dataset):
#
#     n_tasks= len(TASKS)
#
#     #todo change NonLinearModel to LiearModel
#     model = NonLinearModel(n_tasks).to(device_name)
#
#     # model.n_tasks=n_tasks
#     model.n_classes=1
#     train_ratios = get_class_imbalance_ratio(dataset)
#     # ratios = get_class_imbalance_ratio(train_dataset)
#     lr = 1e-4
#
#     learning_rate = 0.001
#     modeldeepchem = LinearModelMultiLabelDeepchem(\
#         model=model,class_imbalance_ratio=train_ratios,loss_aggr_type='sum',\
#         device_name=device_name,mode=model,optimizer_name = 'adam',\
#         learning_rate=learning_rate,batch_size=128)
#
#     return modeldeepchem
#

In [ ]:
# def run_linear_classifier(modeldeepchem,embedding_dataset,embedding_test_dataset,metric,metric_str,nb_epoch = 2):
#     start_time = datetime.now()
#
#     train_scores=[]
#     valid_scores=[]
#     losses=[]
#     for epoch in range(1, nb_epoch+1):
#         loss = modeldeepchem.fit(
#               embedding_dataset,
#               nb_epoch=1,
#               max_checkpoints_to_keep=1,
#               deterministic=False,
#               restore=epoch>1)
#         losses.append(loss)
#
#     modeldeepchem.save_checkpoint()
#     return modeldeepchem,train_scores,valid_scores,losses
